In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('/kaggle/input/quora-question-pairs/train.csv.zip')
test = pd.read_csv('/kaggle/input/quora-question-pairs/test.csv')
submission = pd.read_csv('/kaggle/input/quora-question-pairs/sample_submission.csv.zip')

In [4]:
train.head(5)

In [5]:
test.head(5)

In [6]:
plt.style.use('fivethirtyeight')
%matplotlib inline

f,ax=plt.subplots(1,figsize=(8,8))
train['is_duplicate'].value_counts().plot.pie(autopct='%1.1f%%',ax=ax,shadow=True)

In [7]:
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm

In [8]:
try:    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [9]:
model_name = 'jplu/tf-xlm-roberta-base'
n_epochs = 10
max_len = 70

# Our batch size will depend on number of replicas
batch_size = 16 * strategy.num_replicas_in_sync

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
train.isnull().sum()

In [12]:
test.isnull().sum()

In [13]:
train = train.dropna(axis = 0)

In [14]:
test = test.dropna(axis = 0)

In [15]:
train.isnull().sum()

In [16]:
train.shape

In [17]:
#train = train.sample(frac = 0.85)
train.shape

In [18]:
test = test.sample(frac = 0.01)
test.shape

In [19]:
from imblearn.over_sampling import RandomOverSampler 


In [20]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(train[['question1', 'question2']].values, train.is_duplicate.values)
print(X_over.shape)
print(y_over.shape)

In [21]:
%%time

train_text = X_over.tolist()
test_text = test[['question1', 'question2']].values.tolist()

# Now, we use the tokenizer we loaded to encode the text
train_encoded = tokenizer.batch_encode_plus(
    train_text,
    pad_to_max_length=True,
    max_length=max_len)

test_encoded = tokenizer.batch_encode_plus(
    test_text,
    pad_to_max_length=True,
    max_length=max_len)

In [22]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(
    train_encoded['input_ids'], y_over, 
    test_size=0.15, random_state=2020)

x_test = test_encoded['input_ids']

In [23]:
print((y_valid == 1).sum())
print((y_valid == 0).sum())

In [24]:
auto = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(auto)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(batch_size)
    .cache()
    .prefetch(auto)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(batch_size)
)

In [25]:
from tensorflow.keras.optimizers import SGD

In [26]:
from tensorflow.keras.layers import LSTM, Dropout, Reshape


In [27]:
with strategy.scope():
    # First load the transformer layer
    transformer_encoder = TFAutoModel.from_pretrained(model_name)

    # This will be the input tokens 
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")

    # Now, we encode the text using the transformers we just loaded
    sequence_output = transformer_encoder(input_ids)[0]

    # Only extract the token used for classification, which is <s>
    cls_token = sequence_output[:, 0, :]
    
    rs = Reshape((1, 768), input_shape=(12,))(cls_token)
    
    lst1 = LSTM(50, input_shape=(1, 768), return_sequences=True)(rs)
    
    lst2 = LSTM(20)(lst1)
    
    drp = Dropout(0.5)(lst2)
    
    # Finally, pass it through a 3-way softmax, since there's 3 possible laels
    out = Dense(2, activation='softmax')(drp)

    # It's time to build and compile the model
    model = Model(inputs=input_ids, outputs=out)
    model.compile(
        Adam(lr=1e-5), 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )
model.summary()

In [28]:
n_steps = len(x_train) // batch_size

train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=20
)

In [29]:
test_preds = model.predict(test_dataset, verbose=1)
test['is_duplicate'] = test_preds.argmax(axis=1)

In [30]:
test.sample(n = 30)

In [38]:
vall_loss = train_history.history['val_loss']
vall_acc = train_history.history['val_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(vall_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, vall_loss, 'r--')
plt.plot(epoch_count, vall_acc, 'b-')
plt.legend(['Validation Loss', 'Validation Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.show();

In [39]:
training_loss = train_history.history['loss']
training_acc = train_history.history['accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, training_acc, 'b-')
plt.legend(['Training Loss', 'Training Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.show();

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [33]:
yy = model.predict(x_valid, verbose=0)
print(yy)

In [34]:
yp = np.squeeze(yy)
print(yp.shape)
yhat_classes = np.argmax(yy, axis=1)

print(yhat_classes)

In [35]:

accuracy = accuracy_score(y_valid, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(y_valid, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(y_valid, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(y_valid, yhat_classes)
print('F1 score: %f' % f1)

In [36]:
cf_matrix = confusion_matrix(y_valid, yhat_classes)

print(cf_matrix)

In [37]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(cf_matrix, range(2), range(2))
plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, cmap='Blues', fmt="d") # font size

plt.show()